In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *
from xfem.lsetcurv import *
from vizualization import *

importing ngsxfem-2.1.2405


In [2]:
from ngsolve import *
from math import pi

# Analytische Geschwindigkeit (als GridFunction oder CoefficientFunction)
uexact = CoefficientFunction((
    sin(pi*x) * cos(pi*y),   # u(x,y)
    -cos(pi*x) * sin(pi*y)   # v(x,y)
))
pexact = sin(pi*x) * sin(pi*y)
# Laplace von uexact (Komponentenweise)
lapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2
lapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2  # Minuszeichen kommt von v

# Gradient von p
dp_dx = pi * cos(pi*x) * sin(pi*y)
dp_dy = pi * sin(pi*x) * cos(pi*y)

# Gesamte rechte Seite f = -Δu + ∇p
f = CoefficientFunction((
    -lapu_x + dp_dx,
    -lapu_y + dp_dy
))

r = sqrt(x**2 + y**2)
levelset = r - 1
#f= CF((0,x))

#uexact = CoefficientFunction((    -r * (1 - r**2) * y, r * (1 - r**2) * x))

# Analytischer Druck
#pexact = (r + 3 - 15 * r**2) * y
u1 = -4*y * (1 - x**2 - y**2)
u2 = 4*x*(1 - x**2 - y**2)
uexact = CoefficientFunction((u1, u2))

pexact = sin(x)*cos(y)

# Laplace-Anteil
lapu1 = 32*y
lapu2 = -32*x


# Gradient von p
dpdx = cos(x) * cos(y)
dpdy = -sin(x) * sin(y)

#pexact = 0
#dpdx = 0
#dpdy = 0
f = CoefficientFunction((
    -lapu1 - dpdx,
    -lapu2 - dpdy
))

In [ ]:
l2erroru = []
l2errorp = []
for maxh in [0.5,0.25,0.125]:
    for order in [4,5,6,7]:
        # Stabilization parameter for ghost-penalty
        gamma_stab = 100
        # Stabilization parameter for Nitsche
        lambda_nitsche = 10 * order * order

        beta0 = 10 * order**2
        beta1 = 1
        beta2 = 10 * order**2
        beta3 = 1
        square = SplineGeometry()
        square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
        ngmesh = square.GenerateMesh(maxh=maxh)
        mesh = Mesh(ngmesh)
        #lsetp1.Set(levelset)

        lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.1,discontinuous_qn=True)# Higher order level set approximation 
        deformation = lsetmeshadap.CalcDeformation(levelset)

        lsetp1 = GridFunction(H1(mesh,order=1,autoupdate=True),autoupdate=True)
        InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
        Draw(lsetp1, mesh, f"levelset_maxh{maxh}")
        ci = CutInfo(mesh, lsetp1)
        hasneg = ci.GetElementsOfType(HASNEG)
        neg = ci.GetElementsOfType(NEG)
        hasif = ci.GetElementsOfType(IF)
        haspos = ci.GetElementsOfType(HASPOS)
        ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
        patch_elements = ci.GetElementsOfType(ANY)
        #cut_elements = ci.GetElementsOfType(HASIF)  


        interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
        interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
        

        
        h = specialcf.mesh_size
        n = Normalize(grad(lsetp1))

        # integration domains:
        dx = dCut(lsetp1, NEG, definedonelements=hasneg,deformation=deformation)
        ds = dCut(lsetp1, IF, definedonelements=hasif,deformation=deformation)

        dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
        dw_int = dFacetPatch(definedonelements=interior_facets,deformation=deformation)
        dw_interface = dFacetPatch(definedonelements=interface_facet_set, deformation=deformation)
        V = VectorH1(mesh, order=order,dgjumps=True)
        V = Compress(V, GetDofsOfElements(V,hasneg))
        Q = H1(mesh, order=order-1)
        Q = Compress(Q, GetDofsOfElements(Q,hasneg))
        Z = NumberSpace(mesh)
        X = V*Q*Z
        (u,p,z),(v,q,y) = X.TnT()
        gfu = GridFunction(X)

        def jump(f):
            return f - f.Other()
        def jumpn(f):
            n = Normalize(grad(lsetp1))
            return Grad(f)*n - Grad(f).Other()*n

        a = BilinearForm(X)
        stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
        a += stokes
        a += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #nitzshe stabilization
        a += -(q*n * u + p*n * v) * ds
        a += p*y *dx + q *z*dx
        a += beta2* InnerProduct(jump(Grad(u)), jump(Grad(v))) * dw_interface #velocity ghost penalty
        a += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #pressure ghost penalty
        a.Assemble()

        rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
        rhs += InnerProduct(f, v)* dx
        #rhs += -(Grad(v)*n * uexact) * ds + gamma_stab / h * uexact * v * ds+q*n*uexact *ds
        rhs.Assemble()
        gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec
        
        error_u = sqrt(Integrate( (gfu.components[0] - uexact) ** 2*dx, mesh ))
        print("||u_h - u_exact||_L2 =", error_u)
        error_p = sqrt(Integrate( (gfu.components[1] - pexact) ** 2*dx, mesh ))
        print("||p_h - p_exact||_L2 =", error_p ,"order=", order, "maxh=", maxh)

        l2erroru.append((maxh, order, error_u))
        l2errorp.append((maxh, order, error_p))
        #Draw(gfu.components[0], mesh, "u")
        #Draw(gfu.components[1], mesh, "p")

        #DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
        #DrawDC(lsetp1,gfu.components[1], 0, mesh, "p")

print("L2 errors velocity:", triplet_table(l2erroru, 0,1))
print("L2 errors pressure:", triplet_table(l2errorp, 0,1))


||u_h - u_exact||_L2 = 0.5772220460391916
||p_h - p_exact||_L2 = 0.8420203273413139 order= 4 maxh= 0.5
||u_h - u_exact||_L2 = 0.42737572346101826
||p_h - p_exact||_L2 = 1.1866709537656144 order= 5 maxh= 0.5
||u_h - u_exact||_L2 = 0.6205028153586295
||p_h - p_exact||_L2 = 1.2813907058135934 order= 6 maxh= 0.5
||u_h - u_exact||_L2 = 0.6953173757098833
||p_h - p_exact||_L2 = 1.6989705118049323 order= 7 maxh= 0.5
||u_h - u_exact||_L2 = 0.0016601721708027033
||p_h - p_exact||_L2 = 0.014575967642531265 order= 4 maxh= 0.25
||u_h - u_exact||_L2 = 0.00018049449383518708
||p_h - p_exact||_L2 = 0.0022107048241326852 order= 5 maxh= 0.25
||u_h - u_exact||_L2 = 2.7545132514806313e-05
||p_h - p_exact||_L2 = 0.0005696160350468138 order= 6 maxh= 0.25
||u_h - u_exact||_L2 = 3.5047913737470435e-06
||p_h - p_exact||_L2 = 4.842534040760953e-05 order= 7 maxh= 0.25
||u_h - u_exact||_L2 = 2.77967027876805e-05
||p_h - p_exact||_L2 = 0.0007959040851410629 order= 4 maxh= 0.125
||u_h - u_exact||_L2 = 2.1645140701

In [4]:
from numpy import log2
